In [1]:
import tensorflow as tf

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
layer = tf.keras.layers.Dense(100)
layer

In [5]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
layer

In [6]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [7]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.47415346,  0.5440895 ,  0.10530299, -0.575803  , -0.26228416,
          0.08024079, -0.39910436,  0.30063707, -0.522436  ,  0.14791286],
        [ 0.04455304, -0.4522177 ,  0.42560166,  0.4162032 , -0.31802833,
          0.07634228, -0.20477423,  0.55808026, -0.51320314, -0.34700218],
        [ 0.2024073 ,  0.4933836 ,  0.08635998,  0.14720398, -0.08694273,
         -0.11486638,  0.26944017, -0.15835047, -0.55342525,  0.54054564],
        [-0.41248298,  0.1930368 ,  0.4198864 ,  0.18077159, -0.25540078,
         -0.5737455 ,  0.0452435 ,  0.25236148, -0.1154083 ,  0.6197712 ],
        [-0.0541802 , -0.37932095, -0.21270528, -0.3564209 , -0.42919567,
         -0.26172426, -0.07271248,  0.12668502,  0.40910834, -0.24517924]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [8]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.47415346,  0.5440895 ,  0.10530299, -0.575803  , -0.26228416,
          0.08024079, -0.39910436,  0.30063707, -0.522436  ,  0.14791286],
        [ 0.04455304, -0.4522177 ,  0.42560166,  0.4162032 , -0.31802833,
          0.07634228, -0.20477423,  0.55808026, -0.51320314, -0.34700218],
        [ 0.2024073 ,  0.4933836 ,  0.08635998,  0.14720398, -0.08694273,
         -0.11486638,  0.26944017, -0.15835047, -0.55342525,  0.54054564],
        [-0.41248298,  0.1930368 ,  0.4198864 ,  0.18077159, -0.25540078,
         -0.5737455 ,  0.0452435 ,  0.25236148, -0.1154083 ,  0.6197712 ],
        [-0.0541802 , -0.37932095, -0.21270528, -0.3564209 , -0.42919567,
         -0.26172426, -0.07271248,  0.12668502,  0.40910834, -0.24517924]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

In [11]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])
    
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [12]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [13]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer_1/kernel:0']


In [15]:
class ResnetIndentitiyBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIndentitiyBlock, self).__init__()
        filters1, filters2, filters3 = filters
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding="same")
        self.bn2b = tf.keras.layers.BatchNormalization()
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)

block = ResnetIndentitiyBlock(1, [1, 2, 3])

In [17]:
block.layers

In [18]:
len(block.variables)

18

In [22]:
block.summary()

Model: "resnet_indentitiy_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization (BatchN  multiple                 4         
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_1 (Batc  multiple                 8         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_2 (Batc  multiple     

In [23]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Conv2D(2, 1, padding="same"),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Conv2D(3, (1, 1)),
                            tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [24]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_3 (Batc  (None, None, None, 1)    4         
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 2)     4         
                                                                 
 batch_normalization_4 (Batc  (None, None, None, 2)    8         
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, None, None, 3)     9         
                                                                 
 batch_normalization_5 (Batc  (None, None, None, 3)    1